## Imports and Config

In [ ]:
import pathlib
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import timm
from albumentations.pytorch import ToTensorV2
from fastai.vision.all import *
from sklearn.preprocessing import MinMaxScaler


pathlib.WindowsPath = pathlib.PosixPath

In [ ]:
O_TEST_DATASET = Path("./data/test/")
R_TEST_DATASET = Path("./data/test_256/")
BLEND_MODELS_DIR = Path("./models/vitbasefold_vhaugs/")
TTA_PREDICT = False

model_names = [path.name for path in list(BLEND_MODELS_DIR.rglob("*.pkl"))]
test_files = [str(path) for path in sorted(list(R_TEST_DATASET.iterdir()))]

In [ ]:
# needed for models with albumentations transform
class AlbumentationsTransform(DisplayedTransform):
    split_idx,order=0,2
    def __init__(self, train_aug): store_attr()
    
    def encodes(self, img: PILImage):
        aug_img = self.train_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

## Submission

In [ ]:
all_preds = []
for model in model_names:
    path = BLEND_MODELS_DIR / model
    learn = load_learner(path, cpu=False)
    test_dl = learn.dls.test_dl(test_files)

    if not TTA_PREDICT:
        preds, _ = learn.get_preds(dl=test_dl)
    else:
        preds, _ = learn.tta(dl=test_dl, beta=0.75, n=15)

    all_preds.append(MinMaxScaler().fit_transform(preds.numpy()))
    
all_preds = np.array(all_preds).mean(0)

In [ ]:
subm = pd.DataFrame(
    {
        "image_name": [
            str(path.name) for path in sorted(list(O_TEST_DATASET.iterdir()))
        ],
        "label_id": all_preds.argmax(-1),
    }
)
subm.to_csv("submissoins/subm1.csv", index=False, sep="\t")
